<a href="https://colab.research.google.com/github/keaneflynn/LOTIC/blob/main/transferLearning_mobilenet_ssd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Setting working directory to Google Drive** 

In [11]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train/ #create drive directory with necessary model files, tfrecords files, model training file, and adjusted pipeline.config file

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train


In [12]:
%ls * #verify drive is mounted and check directory

model_main.py

model_out:
checkpoint
eval_0/
events.out.tfevents.1617645861.451aeec17bcc
events.out.tfevents.1617646072.451aeec17bcc
events.out.tfevents.1617646574.451aeec17bcc
events.out.tfevents.1617647886.da411f64fc3e
events.out.tfevents.1617651212.da411f64fc3e
events.out.tfevents.1617831506.d64423645b93
events.out.tfevents.1617855975.440a84075a09
graph.pbtxt
model.ckpt-0.data-00000-of-00001
model.ckpt-0.index
model.ckpt-0.meta
model.ckpt-1155.data-00000-of-00001
model.ckpt-1155.index
model.ckpt-1155.meta
model.ckpt-1695.data-00000-of-00001
model.ckpt-1695.index
model.ckpt-1695.meta
model.ckpt-2954.data-00000-of-00001
model.ckpt-2954.index
model.ckpt-2954.meta
model.ckpt-629.data-00000-of-00001
model.ckpt-629.index
model.ckpt-629.meta

models:
AUTHORS     community/       ISSUES.md  official/  README.md
CODEOWNERS  CONTRIBUTING.md  LICENSE    orbit/     research/

model_specs:
config_input/  pipeline.config


##**Verifying GPU is mounted and operating**

In [3]:
%tensorflow_version 1.x #set to tensorflow v1 for use of quantized ssd mobilenet model (not currently supported by tensorflow 2)
import tensorflow as tf
gpu_mounted = tf.test.gpu_device_name()
if gpu_mounted == '/device:GPU:0':
  print('GPU successfully mounted')
else:
  print('No GPU detected')

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x #set to tensorflow v1 for use of quantized ssd mobilenet model (not currently supported by tensorflow 2)`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
GPU successfully mounted


##**Clone into the Tensorflow model repository**

In [4]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

/content


##**Install dependencies for model training and test tensorflow model builder**

In [6]:
!pip install tf_slim

!apt-get update && install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install Cython contextlib2 pillow lxml matplotlib

!pip install pycocotools

!pip install lvis

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

     |████████████████████████████████| 358kB 7.9MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/

##**Train the model!** 
This can take a very long time depending on quantity and size of training/test images, Google Colab only allows for 12 hour use of their online GPU then will boot you. This code will checkpoint your training progress and the training can be picked up at any time, however 100,000 training steps should be sufficient for most object detection applications. In the meantime, go walk outside, smell some flowers, and look for some birds (birds are sick).

In [7]:
!python /content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train/model_main.py \
    --pipeline_config_path=/content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train/model_specs/pipeline.config \
    --model_dir=/content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train/model_out \
    --alsologtostderr \
    --num_train_steps=100000 \
    --num_eval_steps=50 

W0408 04:25:47.426803 139986994775936 model_lib.py:813] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 100000
I0408 04:25:47.427086 139986994775936 config_util.py:552] Maybe overwriting train_steps: 100000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0408 04:25:47.427213 139986994775936 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0408 04:25:47.427299 139986994775936 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0408 04:25:47.427383 139986994775936 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0408 04:25:47.427495 139986994775936 model_lib.py:829] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_

##**Export your model into frozen tensorflow protobuf graphs (.pb files)**
The second export will be the one we will convert into our quantized .tflite file, the first one is a standard Tensorflow protobuf file.

In [ ]:
%cd /content/models/research

import re
import numpy as np

output_directory = './fine_tuned_model'
tflite_directory = './fine_tuned_model/tflite'

lst = os.listdir('/content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train/model_out/')
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join('/content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train/model_out/', last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train/model_specs/pipeline.config \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train/model_specs/pipeline.config \
    --output_directory={tflite_directory} \
    --trained_checkpoint_prefix={last_model_path}

/content/models/research
/content/drive/MyDrive/ObjectDetection/juvenileSalmonids_train/model_out/model.ckpt-100000
Instructions for updating:
Please use `layer.__call__` method instead.
W0405 06:39:50.873871 139687119705984 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0405 06:39:52.960301 139687119705984 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0405 06:39:52.997013 139687119705984 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0405 06:39:53.032933 139687119705984 convolutional_

##**Export our final quantized .tflite file!** 
We can later convert this to an _edgetpu.tflite file using docker (see github repo) that is compatible for use on a Raspberry Pi with a Google Coral TPU device.

In [ ]:
!tflite_convert \
  --output_file='{output_directory}/juvSalmonidv1.0_model.tflite' \
  --graph_def_file=/content/models/research/fine_tuned_model/tflite/tflite_graph.pb \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

2021-04-05 06:40:36.745798: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-05 06:40:36.751310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-05 06:40:36.751861: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2021-04-05 06:40:36.752114: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-04-05 06:40:36.753530: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-04-05 06:40:36.755100: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-